In [1]:
from __future__ import annotations
import openai
import urllib
import pandas as pd

urls = [
    "https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/",
    "https://www.inoue-kobo.com/aws/selenium-serverless/",
    "https://www.inoue-kobo.com/aws/aws-service-summary/",
    "https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/",
    "https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/",
]

summary_df = pd.DataFrame(urls, columns=["url"])
pd.set_option("display.max_colwidth", None)

## 単純に apply するだけ

In [2]:
def get_summary(url: str) -> str | None:
    res_web = urllib.request.urlopen(url)  # type: ignore
    content = res_web.read().decode("utf-8")

    res_openai = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "以下はWebサイトの内容です。HTMLタグを削除した上で、150文字以内で要約してください。",
            },
            {"role": "user", "content": content},
        ],
    )

    return res_openai.choices[0].message.content

In [3]:
summary_df["summary"] = summary_df["url"].apply(get_summary)

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,INOUE-KOBO.COMの記事では、OpenAI API Embeddingsのdim指定の実装方法が紹介されています。新しいモデルtext-embedding-3では、出力の次元数を指定できるようになりました。記事ではAPIを使用して次元数を指定する方法やPCAを使用した次元削減の方法などが詳細に説明されています。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,Webサイトでは、SeleniumをAWS Lambdaでサーバーレスに動かす方法について解説しています。Dockerコンテナやコードサンプルを利用し、AWS CDKで実行する手順が記載されています。デプロイ方法やテスト方法も詳細に説明されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,このWebサイトは、AWSサービス一覧をNotionデータベースにまとめる方法について解説しています。AWS公式のサービス一覧を取得し、Notionデータベースに登録する手順やNotion AIを利用した説明の追加方法が記載されています。記事にはPythonコードや参考文献も含まれています。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要でDuckDuckGoとLangChainを使用して質問応答ボットを作る方法についての記事。LangChainのAgent機能を使い、Google検索APIの代わりにDuckDuckGoを利用。LangSmithとの連携も紹介。Pythonコードや実行例も掲載。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,このWebサイトは、ローカルに保存した英語のPDFに日本語で質問応答するGradioのデモンストレーションを提供しています。PDFを読み込み、OpenAIのモデルを使用して質問に回答するWebアプリケーションを実装しています。具体的なユースケースや手順、実行結果などが詳細に記載されています。


## async を使う


In [4]:
import asyncio


async def aget_summary(url: str) -> str | None:
    res_web = urllib.request.urlopen(url)  # type: ignore
    content = res_web.read().decode("utf-8")

    res_openai = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "以下はWebサイトの内容です。HTMLタグを削除した上で、150文字以内で要約してください。",
            },
            {"role": "user", "content": content},
        ],
    )

    return res_openai.choices[0].message.content

In [16]:
summary_df["summary"] = await asyncio.gather(*[aget_summary(url) for url in urls])

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,INOUE-KOBO.COMの記事「OpenAI API Embeddingsのdim指定の実装方法を確認した」では、OpenAI APIの新機能であるベクトル表現取得用モデルの次元数指定方法について解説しています。APIで次元数を指定する方法やPCAによる次元削減、コサイン類似度の計算方法などが紹介されています。記事にはPythonのコードも含まれています。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,Webサイトでは、SeleniumをAWS Lambdaでサーバーレスに動かす方法について解説しています。Dockerコンテナやコードサンプルを利用し、AWS CDKで実行する手順が記載されています。デプロイ方法やテスト方法も詳細に説明されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSサービス一覧をNotionデータベースにまとめる方法が記載されたWebページ。AWS公式からサービス一覧を取得し、Notionデータベースに登録する手順やNotion AIを利用して説明を追加する方法が解説されている。記事内にはPythonコードや参考文献も記載されている。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要でDuckDuckGoとLangChainを使用して質問応答ボットを作る方法についての記事が掲載されています。LangChainのAgent機能を利用し、Google検索APIの代わりにDuckDuckGoを使用する手順が詳細に説明されています。LangSmithとの連携も紹介されており、Pythonのソースコードや実行例も提供されています。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,このWebサイトは、Gradioを使用してローカルに保存した英語のPDFに日本語で質問応答するデモンストレーションを提供しています。PDFを読み込み、OpenAIのモデルを使用して質問に回答するWebアプリケーションを実装しています。具体的なユースケースや手順、実行結果が記載されています。


## ThreadPoolExecutorを使う

In [6]:
from concurrent.futures import ThreadPoolExecutor

In [15]:
with ThreadPoolExecutor() as executor:
    summary_df["summary"] = list(executor.map(get_summary, urls))

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,INOUE-KOBO.COMの記事「OpenAI API Embeddingsのdim指定の実装方法を確認した」では、OpenAI APIの新しいモデルtext-embedding-3で出力次元数を指定する方法が紹介されています。記事ではAPIでの次元指定やPCAによる次元削減の方法が示され、結果も示されています。記事の最後には参考文献も掲載されています。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,このWebサイトでは、SeleniumをAWS Lambdaでサーバーレスに動かす方法について詳しく説明されています。Dockerコンテナやコードサンプルも提供されており、AWS CDKを使用して実行例を作成する手順も紹介されています。デプロイ方法やテスト方法も記載されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSサービス一覧をNotionデータベースにまとめる方法が紹介されている。AWS公式のサービス一覧を取得し、Notionデータベースに登録する手順やNotion AIを利用して説明を追加する方法が解説されている。記事内にはPythonコードや参考文献も記載されている。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要でDuckDuckGoとLangChainを使用して質問応答ボットを作る方法についての記事。LangChainのAgent機能を使い、Google検索APIの代わりにDuckDuckGoを利用。LangSmithのログトレース機能も活用。Pythonコードや実行例も掲載。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,このWebサイトは、ローカルに保存した英語のPDFに日本語で質問応答するGradioのデモンストレーションを提供しています。PDFを読み込み、OpenAIのモデルを使用して質問に回答するWebアプリケーションを実装しています。具体的なユースケースや実行手順が記載されています。また、Gradioのソースコードや実行結果も示されています。


## pandarallelを使う

In [8]:
from pandarallel import pandarallel

pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [12]:
summary_df["summary"] = summary_df["url"].parallel_apply(get_summary)

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,INOUE-KOBO.COMの記事では、OpenAI API Embeddingsのdim指定の実装方法が紹介されています。新しいモデルtext-embedding-3では、出力の次元数を指定できるようになりました。記事ではAPIを使用して次元数を指定する方法やPCAを使用した次元削減の方法が紹介されています。記事にはPythonのコード例や結果の画像も含まれています。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,このWebサイトは、SeleniumをAWS Lambdaでサーバーレスに動かす方法についての記事が掲載されています。DockerコンテナやAWS CDKを使用して、SeleniumをHeadlessで実行する方法が紹介されています。記事の内容やコードの説明が詳細に記載されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSサービス一覧をNotionデータベースにまとめる方法が記載されたWebページ。AWS公式のサービス一覧を取得し、Notionデータベースに登録する手順やNotion AIを利用して説明を追加する方法が解説されている。記事内にはPythonコードや参考文献も記載されている。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要でDuckDuckGoとLangChainを使用して質問応答ボットを作る方法についての記事。LangChainのAgent機能を利用し、Google検索APIの代わりにDuckDuckGoを使用。LangSmithとの連携も紹介。Pythonのソースコードや実行例も掲載。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,Webサイトでは、ローカルに保存した英語のPDFに日本語で質問応答するGradioのデモンストレーションが紹介されています。PDFを読み込み、OpenAIのモデルを使用して質問応答を行うWebアプリケーションの実装手順やコードが示されています。具体的なユースケースや実行結果も紹介されています。また、関連するライブラリやツールの情報も提供されています。
